In [ ]:
import requests
import datetime
from dateutil.relativedelta import relativedelta
import re
import psycopg2 as pg2
import Collection

In [ ]:
def regionCreate(nation):

    conn = pg2.connect(host=host,
                       dbname=dbname,
                       user=user,
                       password=password,
                       port=port)

    cur = conn.cursor()

    sql = "insert into nation_region_tbl values "

    continent = Collection.getContinent(nation)
    regions = Collection.getRegionList(nation)

    one_month_ago = datetime.date.today() - relativedelta(months=1)
    year = one_month_ago.strftime('%Y')
    month = one_month_ago.strftime('%m')
    date = year + month

    for regions in regions:
        url = ('https://newsapi.org/v2/everything?'
               'q=' + nation + ' AND (' + Collection.getDiseaseList() + ') AND ' + regions + '&'
               'from=' + year + '-' + month + '-' + '01' + 'T01:00:00&'
               'to=' + year + '-' + month + '-' + '31' + 'T23:59:59&'
               'pageSize=100&'
               'apiKey=' + API_KEY)
        response = requests.get(url)
        totalResult = response.json()['totalResults']

        sql += (
            "('" + date + "', '" + continent + "', '" + nation + "', '" + regions + "', " + str(totalResult) + "),"
        )

    sql = sql[:-1]

    cur.execute(sql)
    conn.commit()

    cur.close()
    conn.close()

    print('------- Regions [ ' + date + ' , ' + nation + ' ] -------')

In [ ]:

def countCreate(nation):

    conn = pg2.connect(host=host,
                       dbname=dbname,
                       user=user,
                       password=password,
                       port=port)

    cur = conn.cursor()

    one_month_ago = datetime.date.today() - relativedelta(months=1)
    year = one_month_ago.strftime('%Y')
    month = one_month_ago.strftime('%m')
    date = year + month

    sql = (
        "select * from nation_mediacontents_tbl "
        "where"
        "   date >= '" + date + "01'" + " and date <= '" + date + "31' and nation = '" + nation + "'"
    )

    cur.execute(sql)
    rows = cur.fetchall()

    diseaseTerms = re.sub("[()]", "", Collection.getDiseaseList())
    diseaseTerms = diseaseTerms.replace(" AND ", " ")
    diseaseTerms = diseaseTerms.split(" OR ")

    diseaseObjectList = []

    for diseaseTerm in diseaseTerms:
        diseaseObjectList.append({"term": diseaseTerm, "count": 0})

    for row in rows:
        article = str(row[4])
        for diseaseObject in diseaseObjectList:
            if diseaseObject['term'].lower() in article.lower():
                diseaseObject['count'] += 1

    continent = Collection.getContinent(nation)

    sql = "insert into nation_count_tbl values "

    for diseaseObject in diseaseObjectList:
        sql += "('" + date + "', '" + continent + "', '" + nation + "', '" + diseaseObject['term'] + "', " + str(diseaseObject['count']) + "),"

    sql = sql[:-1]

    cur.execute(sql)
    conn.commit()

    cur.close()
    conn.close()

    print('------- Count [ ' + date + ' , ' + nation + ' ] -------')
    


In [ ]:
if __name__ == '__main__':
    try:
        today = datetime.date.today()

        nations = Collection.getNationList()

    except Exception as e:
        time = datetime.datetime.now()
        exceptionFile = open(EXCEPTION_PATH + today.strftime('%Y%m%d') + '_' + 'exception.txt', 'a', encoding='utf8')
        exceptionFile.write(str(time) + '\t' + e.__str__() + '\n')